In [98]:
# PyTorch implementation

import numpy as np
import torch
from torch import roll
import timeit


def initialize_x(gridX,gridY):
    x = np.random.rand(gridX,gridY)
    x[:,0] =  0
    x[:,-1] = 0
    x[0,:] = 0
    x[-1,:] = 0

    return x

def jacobi_torch(f):
    newf = 0.25 * ( roll(f,1,0)[1:-1,1:-1] + roll(f,-1,0)[1:-1,1:-1] + roll(f,0,1)[1:-1,1:-1] + roll(f,0,-1)[1:-1,1:-1] )
    return newf

if __name__ == "__main__":

    iterations = 1000
    gridX = 512
    gridY = gridX

    x = initialize_x(gridX,gridY)
    x = torch.from_numpy(x)
    x = x.cuda()

    t1 = timeit.default_timer()
    for i in range(iterations):
      x[1:-1,1:-1] = jacobi_torch(x)

    t2 = timeit.default_timer()

    print("Time to soluton torch:", t2-t1)

Time to soluton torch: 0.16714438300004986


In [3]:
# CuPy implementation

import numpy as np
import cupy as cp
import timeit
import h5py


def initialize_x(gridX,gridY):
    x = np.random.rand(gridX,gridY)
    x[:,0] =  0
    x[:,-1] = 0
    x[0,:] = 0
    x[-1,:] = 0
    return x

def jacobi_cupy(f):
    newf = 0.25 * ( cp.roll(f,1,0)[1:-1,1:-1] + cp.roll(f,-1,0)[1:-1,1:-1] + cp.roll(f,0,1)[1:-1,1:-1] + cp.roll(f,0,-1)[1:-1,1:-1] )
    return newf

if __name__ == "__main__":

    iterations = 1000
    gridX = 512
    gridY = gridX

    x = initialize_x(gridX,gridY)
    outputfile = h5py.File("jacobiIteration.hdf5","w")
    outputfile["initialGrid"] = x
    x = cp.asarray(x)
    t1 = timeit.default_timer()
    for i in range(iterations):
      x[1:-1,1:-1] = jacobi_cupy(x)
    t2 = timeit.default_timer()

    x = cp.asnumpy(x)
    outputfile["newGrid"] = x
    outputfile.close()
    print("Time to soluton CuPy:", t2-t1)

Time to soluton CuPy: 0.36674304799998936
